# Production stage: use files 0-6 for model traning

In [1]:
!ls raw_data


data_gathered_at_0.csv	   data_gathered_at_3.csv  initial_training_data.csv
data_gathered_at_1.csv	   data_gathered_at_4.csv  test_data.csv
data_gathered_at_2.csv	   data_gathered_at_5.csv  validation_test_data.csv
data_gathered_at_2_bk.csv  data_gathered_at_6.csv


In [2]:
!mkdir accum_data

mkdir: cannot create directory ‘accum_data’: File exists


In [3]:
!cp raw_data/initial_training_data.csv accum_data/accumulated_data.csv


In [5]:
!ls -l accum_data/

total 2620
-rw-r--r-- 1 root root 2679360 May 30 12:42 accumulated_data.csv


In [6]:
import pandas as pd
import numpy as np
import os

from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical


raw_data_path = 'raw_data/'
accum_data_path = 'accum_data/'
accum_file_name = accum_data_path+'accumulated_data.csv'

def create_model():
    model = models.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['Emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'Pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


def data_to_tf_data(df):
    image_array, image_label = prepare_data(df)
    images = image_array.reshape((image_array.shape[0], 48, 48, 1))
    images = images.astype('float32')/255
    labels = to_categorical(image_label)
    return images, labels


In [7]:
def add_to_accum_data(nmb):
    accum_data = pd.read_csv(accum_file_name)
    next_data_file = raw_data_path+'data_gathered_at_' + str(nmb)+'.csv'
    add_data = pd.read_csv(next_data_file)
    accum_data = accum_data.append(add_data, ignore_index = True)
    accum_data.to_csv(accum_file_name)


In [8]:
nmb = 0
add_to_accum_data(nmb)


In [9]:
train = pd.read_csv(accum_file_name)
train_images, train_labels = data_to_tf_data(train)

val = pd.read_csv(raw_data_path+'validation_test_data.csv')
val_images, val_labels = data_to_tf_data(val)


model = create_model()
class_weight = dict(zip(range(0, 7), (((train['Emotion'].value_counts()).sort_index())/len(train['Emotion'])).tolist()))
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

df = pd.read_csv(raw_data_path+'test_data.csv')
test_images, test_labels = data_to_tf_data(df)
test_loss, test_acc = model.evaluate(test_images, test_labels)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/12
7/7 [==============================] - 4s 551ms/step - loss: 0.3106 - accuracy: 0.2633 - val_loss: 1.8926 - val_accuracy: 0.2449
Epoch 2/12
7/7 [==============================] - 3s 496ms/step - loss: 0.2972 - accuracy: 0.2826 - val_loss: 2.0590 - val_accuracy: 0.2449
Epoch 3/12
7/7 [==============================] - 3s 487ms/step - loss: 0.2952 - accuracy: 0.2826 - val_loss: 1.8531 - val_accuracy: 0.2449
Epoch 4/12
7/7 [==============================] - 3s 503ms/step - loss: 0.2943 - accuracy: 0.2826 - val_loss: 1.9506 - val_accuracy: 0.2449
Epoch 5/12
7/7 [==============================] - 3s 469ms/step - loss: 0.2898 - accuracy: 0.2826 - val_loss: 1.8896 - val_accuracy: 0.2449
Epoch 6/12
7/7 [==============================] - 3s 493ms/step - loss: 0.2887 - accuracy: 0.2826 - val_loss: 1.9448 - val_accuracy: 0.2449
Epoch 7/12
7/7 [==============================] - 4s 564ms/step - loss: 0.2873 - accuracy: 0.2826 - val_loss: 1.9205 - val_accuracy: 0.2449
Epoch 8/12
7/7 [====

In [10]:
train

,Unnamed: 0,Emotion,Pixels,rand
0,0,0,215 218 222 220 221 226 218 215 216 218 218 21...,7.0
1,1,6,145 146 118 26 15 18 16 18 24 38 57 66 71 78 8...,8.0
2,2,4,2 3 4 9 9 10 9 10 12 17 20 21 22 22 19 17 15 1...,9.0
3,3,3,38 42 35 25 51 33 32 20 29 47 40 28 42 56 61 7...,8.0
4,4,2,32 26 37 68 99 112 117 121 123 126 129 135 141...,7.0
...,...,...,...,...
409,409,5,252 255 254 236 149 149 123 110 106 89 71 65 6...,NaN
410,410,6,16 16 13 22 40 49 63 66 61 42 29 56 57 89 109 ...,NaN
411,411,3,60 58 58 38 44 54 54 71 57 42 41 53 90 53 57 5...,NaN
412,412,6,118 124 109 94 66 11 19 48 84 114 147 164 182 ...,NaN


In [50]:
from tensorflow import keras

model.save("initial_model")


INFO:tensorflow:Assets written to: initial_model/assets


In [51]:
!dvc add initial_model/


Adding...                                                                       
!
Computing file/dir hashes (only done once)            |0.00 [00:00,      ?md5/s]
                                                                                
!
                                                      |0.00 [00:00,       ?it/s]
                                                                                
!
  0%|          |Saving files                          0/4 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |.87pwm7wFFH2XC5gZWfEAyJ.tmp    0.00/17.6k [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.R2yyduqujm2tFfC7MYDRJ8.tmp     0.00/165k [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.ECYChNAkaFUd3CaG3EqdZN.tmp    0.00/3.83M [00:00<?,       ?it/s]
                

In [52]:
!git add initial_model.dvc

In [53]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   initial_model.dvc

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   FinalProj2_production.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	Untitled.ipynb
	accum_data/
	mlruns/



In [54]:
!git add FinalProj2_Initial.ipynb FinalProj2_production.ipynb

In [55]:
!git commit -m "Model trained wit data at_1"

[master fcd9e13] Model trained wit data at_1
 2 files changed, 116 insertions(+), 48 deletions(-)
